<a href="https://colab.research.google.com/github/laurenneal/capstone-visual-neuroscience/blob/main/Feature_Engineering_from_Raw_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import sys
from numpy.ma.core import ceil, floor
from more_itertools import sliced

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
! ls drive/MyDrive/DS6011_Capstone_VisualNeuroscience/Seeded_CNMF/Extracted_Features/

210815_0__20220213T070259RAW_stimulus_data.csv
210815_0_allStacks_mc_mix1_syt_rawExtracts


## Functions
This section contains functions that can be used on datasets for feature engineering

In [3]:
raw_df = pd.read_csv('drive/MyDrive/DS6011_Capstone_VisualNeuroscience/Seeded_CNMF/Extracted_Features/210815_0__20220213T070259RAW_stimulus_data.csv', index_col=['stack','label','roi','frame']).drop(columns='Unnamed: 0')
raw_df

filename  ...     stim3
stack label roi frame                                                     ...          
1     tm2   1   0      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...  0.110063
                1      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...  0.000000
                2      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...  0.000000
                3      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...  0.030790
                4      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...  0.122134
...                                                                  ...  ...       ...
10    tm4   15  5508   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...  0.000000
                5509   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...  0.089182
                5510   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...  0.019588
                5511   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...  0.000000
                5512   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...  0.000000

[826950 rows x 5 columns]

## Train Test Split on Stacks

In [6]:
def trainTestSplit(df, splt = [.7,.2,.1]):
  # splits on stack
  # split is the train test valuidation split (proportions given in that order)
  if sum(splt) < 0.9999 or sum(splt) > 1.0001:
    sys.exit('Splt must add to 1')
  
  num_stacks = raw_df.index.unique(level='stack').max()

  tr_ind = int(ceil(num_stacks*splt[0]))
  ts_ind = int(ceil(num_stacks*splt[1])) + tr_ind

  ar = np.arange(num_stacks)
  np.random.shuffle(ar)
  ar = ar+1 # stack id's start from 1

  train = ar[:tr_ind]
  test = ar[tr_ind:ts_ind]
  val = ar[ts_ind:]

  for i in train:
    df.loc[i,'training'] = 'train'

  for i in test:
    df.loc[i,'training'] = 'test'

  for i in val:
    df.loc[i,'training'] = 'validate'

  return(df)

In [7]:
# df_temp = trainTestSplit(raw_df,splt=[.5,.3,.3]) # should give error
df_temp = trainTestSplit(raw_df,splt=[.5,.3,.2])
df_temp.sample(15)

/usr/local/lib/python3.7/dist-packages/ipykernel/ipkernel.py:208: PerformanceWarning: indexing past lexsort depth may impact performance.
  res = shell.run_cell(code, store_history=store_history, silent=silent)


,,,,filename,resp,stim1,stim2,stim3,training
stack,label,roi,frame,,,,,,
8,tm4,15,2737,210815_0_8_stackRaw_mc_mix1_syt_result_2022021...,0.112702,0.465076,42.407613,2.516894e-02,train
5,tm2,5,2681,210815_0_5_stackRaw_mc_mix1_syt_result_2022021...,-0.045210,-0.120886,-27.332576,8.901113e-02,test
1,tm2,5,3500,210815_0_1_stackRaw_mc_mix1_syt_result_2022021...,0.153326,-0.287272,-72.817127,2.414466e-10,train
8,t5,13,2082,210815_0_8_stackRaw_mc_mix1_syt_result_2022021...,0.001151,0.149661,131.459346,6.468570e-03,train
9,tm2,14,1803,210815_0_9_stackRaw_mc_mix1_syt_result_2022021...,0.271802,-0.342791,0.000000,0.000000e+00,validate
10,tm1,7,3640,210815_0_10_stackRaw_mc_mix1_syt_result_202202...,0.689308,-0.606865,114.422374,2.081311e-02,validate
7,tm4,9,1730,210815_0_7_stackRaw_mc_mix1_syt_result_2022021...,0.087415,0.668403,117.957688,1.188714e-02,train
5,tm4,8,667,210815_0_5_stackRaw_mc_mix1_syt_result_2022021...,0.391991,0.367158,171.236042,1.088571e-01,test
3,tm4,8,2203,210815_0_3_stackRaw_mc_mix1_syt_result_2022021...,0.438915,0.257484,128.820407,4.360939e-03,train


## Combine Frames into Temporal Chunks

In [8]:
# select frame size
temporal_period_length = 200

In [11]:

def groupFrames(df, period):
  # inputs are data frame and temporal period length
  # df needs to be sorted correctly, the indexing should take care of that

  # check that all rois are the same size
  # if this is a problem we can write a more computationally heavy workthrough
  if (raw_df.groupby(['stack','roi']).count()['filename'].max() != raw_df.groupby(['stack','roi']).count()['filename'].min()):
    sys.exit('ROIs need to be the same length')

  num_stacks = raw_df.index.unique(level='stack').max()
  num_roi = raw_df.index.unique(level='roi').max()

  # get first label
  lbl_1 = raw_df.head(1).index.values[0][1]
  # use it to find the number of frames in a roi
  num_frames = raw_df.loc[(1,lbl_1,1),'filename'].count()
  # how many groups will ther be in each roi?
  num_pds = int(floor(num_frames/period))
  # how many left over
  remainder_pds = num_frames%period

  # iterate a column
  lst = [[li + 1] * period for li in range(0,num_pds)]
  lst = [li for sublist in lst for li in sublist] # https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-a-list-of-lists
  lst = lst + [lst[-1] + 1] * remainder_pds
  lst

  new_col = np.array(lst * num_stacks * num_roi)
  
  df['frame_group'] = new_col

  return(df)

In [12]:
df_temp2 = groupFrames(raw_df,temporal_period_length)
df_temp2

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


filename  ...  frame_group
stack label roi frame                                                     ...             
1     tm2   1   0      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...            1
                1      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...            1
                2      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...            1
                3      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...            1
                4      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...            1
...                                                                  ...  ...          ...
10    tm4   15  5508   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...           28
                5509   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...           28
                5510   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...           28
                5511   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...           28
                5512   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...           28

[826950 rows x 7 columns]

In [22]:
def stimResponse(stim, response):
  # takes a stim column and response column and returns two new feature column
  # PR is Positive Response
  # NR is Negative Response
  # PS is Positive Stimulus
  # NS is Negative Stimulus

  PR = response > 0
  NR = response <= 0
  PS = stim > 0
  NS = stim <= 0

  PR = PR.rename('PR')
  NR = NR.rename('NR')
  PS = PS.rename('PS')
  NS = NS.rename('NS')

  # combine into df
  df = pd.concat([stim, response, PR,NR,PS,NS], axis=1)

  # get min and max values for normalizing
  S_min = stim.min()
  S_max = stim.max()
  R_min = response.min()
  R_max = response.max()

  # normalize columns (should return absolute values) # not working!!!!!
  # first stimulus
  # df = df.apply(lambda x: x[0]/S_max if x['PS'] else x[0]/S_min, axis=0)
  df.iloc[4,0] = df.iloc[4,0]/(S_max)
  df[df.NS].iloc[:,0] = df[df.NS].iloc[:,0].divide(S_min) # for some reason it looks like it's dividing the entire column by S_min, not just the filtered columns

  # now response
  df[df.PR].iloc[:,1] = df[df.PR].iloc[:,1].divide(R_max)
  df[df.NR].iloc[:,1] = df[df.NR].iloc[:,1].divide(R_min)

  # multiply stim and resp
  df['relation'] = df.iloc[:,0] * df.iloc[:,1]
  df.loc[(df['PR'] & df['PS']),'relation_type'] = 'PRPS'
  df.loc[(df['NR'] & df['NS']),'relation_type'] = 'NRNS'
  df.loc[(df['PR'] & df['NS']),'relation_type'] = 'PRNS'
  df.loc[(df['NR'] & df['PS']),'relation_type'] = 'NRPS'

  
  # return(df)
  return(df[['relation','relation_type']])

In [26]:
# df_temp_t2 = stimResponse(df_temp_t['stim1'],df_temp_t['resp'])
# df_temp_t2#[df_temp_t2['NR']].iloc[:,0]
# df_temp_t2[['relation','relation_type']]
df_temp_3 = raw_df
df_temp_3[['stim1_rel', 'stim1_rel_type']] = stimResponse(raw_df['stim1'],raw_df['resp'])
df_temp_3

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


filename  ...  stim1_rel_type
stack label roi frame                                                     ...                
1     tm2   1   0      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...            NRNS
                1      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...            PRNS
                2      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...            PRNS
                3      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...            PRNS
                4      210815_0_1_stackRaw_mc_mix1_syt_result_2022021...  ...            PRPS
...                                                                  ...  ...             ...
10    tm4   15  5508   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...            PRPS
                5509   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...            PRNS
                5510   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...            PRNS
                5511   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...            PRNS
                5512   210815_0_10_stackRaw_mc_mix1_syt_result_202202...  ...            PRNS

[826950 rows x 9 columns]

In [17]:
df_temp_t = raw_df.sample(50)
# df_temp

In [192]:
# df_temp_t2[(df_temp_t2.PR == True) and (df_temp_t2.PS == True)]#['type'] = 'PRPS'
df_temp_t2.loc[(df_temp_t2['PR'] & df_temp_t2['PS']),'type'] = 'PRPS'
df_temp_t2.loc[(df_temp_t2['NR'] & df_temp_t2['NS']),'type'] = 'NRNS'
df_temp_t2.loc[(df_temp_t2['PR'] & df_temp_t2['NS']),'type'] = 'PRNS'
df_temp_t2.loc[(df_temp_t2['NR'] & df_temp_t2['PS']),'type'] = 'NRPS'
df_temp_t2

,,,,stim1,resp,PR,NR,PS,NS,type
stack,label,roi,frame,,,,,,,
5,tm2,14,207,0.480511,-0.234109,False,True,True,False,NRPS
8,tm1,7,3580,-0.144492,0.354413,True,False,False,True,PRNS
6,tm2,10,1095,0.003241,1.768867,True,False,True,False,PRPS
7,tm9,6,3292,-0.146369,-0.189285,False,True,False,True,NRNS
2,tm2,11,4106,-0.301201,0.740846,True,False,False,True,PRNS
9,tm4,15,622,0.053877,0.469881,True,False,True,False,PRPS
7,tm2,10,131,-0.102012,0.244015,True,False,False,True,PRNS
8,tm2,4,1275,0.091965,0.312462,True,False,True,False,PRPS
2,tm2,1,3189,0.163293,-0.049980,False,True,True,False,NRPS


In [68]:
10001%10

1